In [1]:
import sys, os, os.path
import glob

In [2]:
import scipy as sp
import numpy as np

import matplotlib
import matplotlib.pyplot as pp

In [3]:
import yt
from yt.frontends.boxlib.data_structures import AMReXDataset

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
# Try Cori's scratch... otherwise the user will have to manually input the data root
scratch_env_key = "SCRATCH"
if scratch_env_key in os.environ.keys():
    data_root = os.environ[scratch_env_key]
else:
    data_root = ""
    print(RuntimeWarning("`SCRATCH` is not an environment variable => data_root is empty"))

`SCRATCH` is not an environment variable => data_root is empty


In [6]:
data_dir  = "."
data_path = os.path.join(data_root, data_dir)

In [7]:
step     = 1
n_fill   = 5
prefix   = "plt"
file_fmt = prefix + "{:0" + str(n_fill) + "d}"

In [8]:
data_glob  = os.path.join(data_path, prefix + "*")
data_files = glob.glob(data_glob)
data_files.sort()

In [9]:
def check_sequential(files_sorted, step):
    sequential = True
    for i in range(len(files_sorted)):
        c_file = file_fmt.format(i*step)
        c_path = os.path.join(data_path, c_file)

        missing = list()
        if c_path not in files_sorted:
            missing.append(c_path)
            sequential = False

    return sequential, missing

In [10]:
check_sequential(data_files, step)

(False, ['./plt00011'])

In [11]:
class SoA:
    _pref = "particle_"
    _pos  = "position_"
    _vel  = "vel"
    _mass = "mass"
    def __init__(self, data):
        str_pos = self._pref+self._pos
        self.px = np.array(data[str_pos + "x"])
        self.py = np.array(data[str_pos + "y"])
        self.pz = np.array(data[str_pos + "z"])

        str_vel = self._pref+self._vel
        self.vx = np.array(data[str_vel + "x"])
        self.vy = np.array(data[str_vel + "y"])
        self.vz = np.array(data[str_vel + "z"])

        self.mass = np.array(data[self._pref + self._mass])

        
    def __str__(self):
        return "{pos:"  + str(self.px) + "," + str(self.py) + "," + str(self.pz) + \
              "; vel:"  + str(self.vx) + "," + str(self.vy) + "," + str(self.vz) + \
              "; mass:" + str(self.mass) + "}"


    def __repr__(self):
        return str(self)



class Particle:
    def __init__(self, px, py, pz, vx, vy, vz, mass):
        self.pos = np.array([px, py, pz])
        self.vel = np.array([vx, vy, vz])
        self.mass = mass


    def __str__(self):
        return "P(" + str(self.pos) + "," + str(self.vel) + "," + str(self.mass) + ")"


    def __repr__(self):
        return str(self)


class AoS:
    def __init__(self, amrex_data):
        self.particles = list()
        soa = SoA(amrex_data)
        
        data = zip(soa.px, soa.py, soa.pz, soa.vx, soa.vy, soa.vz, soa.mass)
        for elt in data:
            self.particles.append(Particle(* elt))


In [13]:
ds = AMReXDataset(data_files[-1])

yt : [INFO     ] 2019-04-30 17:24:48,310 Parameters: current_time              = 0.0009000000000000002
yt : [INFO     ] 2019-04-30 17:24:48,320 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2019-04-30 17:24:48,321 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2019-04-30 17:24:48,322 Parameters: domain_right_edge         = [1. 1. 1.]


In [14]:
ds.particle_fields_by_type

defaultdict(list,
            {'immersed_boundaries': ['particle_cpu',
              'particle_density',
              'particle_dragx',
              'particle_dragy',
              'particle_dragz',
              'particle_id',
              'particle_mass',
              'particle_omegax',
              'particle_omegay',
              'particle_omegaz',
              'particle_oneOverI',
              'particle_phase',
              'particle_position_x',
              'particle_position_y',
              'particle_position_z',
              'particle_radius',
              'particle_state',
              'particle_velx',
              'particle_vely',
              'particle_velz',
              'particle_volume']})

In [15]:
ad = ds.all_data()
soa = SoA(ad)
aos = AoS(ad)

In [16]:
aos.particles

[P([0.5002341  0.50600852 0.50024773],[ 0.54624255 13.13256495  0.57813139],0.04188790204786391)]

In [17]:
ad["particle_position_x"]

YTArray([0.5002341]) code_length

In [37]:
part_x = np.array(ad["particle_position_x"])

In [38]:
part_x

array([0.49999245])

In [39]:
np.array([2])

array([2])

In [28]:
ad["particle_position_y"][0]

0.500724076053728 code_length

In [29]:
ad["particle_position_z"][0]

0.4999908907580099 code_length

In [30]:
ad["particle_velx"][0]

-0.0755310174836537 dimensionless

In [31]:
ad["particle_vely"][0]

7.240760537280711 dimensionless

In [32]:
ad["particle_velz"][0]

-0.09109241990090886 dimensionless